In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image as PIL_Image
from pynq import Overlay
from pynq import GPIO
from IPython.display import clear_output as clear
import matplotlib.pyplot as plt
import time

picture_path = 'Picture_datatest/DATAtest.jpg'      # 要识别的图片路径
ol = Overlay('/home/xilinx/pynq/overlays/GPIO/design_1.bit')
LED0 = GPIO(GPIO.get_gpio_pin(0),'out')
LED1 = GPIO(GPIO.get_gpio_pin(1),'out')
LED2 = GPIO(GPIO.get_gpio_pin(2),'out')
LED3 = GPIO(GPIO.get_gpio_pin(3),'out')

In [ ]:
# GPIO输出，1输出3.3V，0输出GND。共阳极0亮1灭
def LED_Voltage(val):
    LED0.write(val)
    LED1.write(val)
    LED2.write(val)
    LED3.write(val)

# 摄像头采集并保存图片
def webcampicture(picture_path):
    !fswebcam  --no-banner --save {picture_path} -d /dev/video0 2> /dev/null

# 人脸识别的主体
def faceRecognition(picture_path):
    frame = cv2.imread(picture_path)                                                    # 读取待识别图片
    frameRGB = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)
    clear()                                                                             # 清除上一次拍的照片与识别数据
    plt.imshow(frameRGB)                                                                # 显示图片
    plt.show()
    facesLocate = face_recognition.face_locations(frameRGB)                             # 对摄像头读取的检测人脸
    faceEncoded = face_recognition.face_encodings(frameRGB, facesLocate)                # 进行特征编码
    lab = 'unknow'
    for face_encoding in faceEncoded:                                                   # 遍历检测的人脸和库中读取的图片进行对比，计算其相似度
        matchs = face_recognition.compare_faces(EncodeingList, face_encoding)           # 进行匹配
        distance = face_recognition.face_distance(EncodeingList, face_encoding)         # 计算相似度
        for index, item in enumerate(distance):
            Similarity = 1 - item
            if Similarity > 0.6:
                if matchs[index]:
                    lab = 'name:{}; Similarity:{}'.format(classNames[index], Similarity)  # 得到匹配到的图片名称与相似度值
                    break
        print(lab)                                                                        # 输出识别信息
        if lab is 'unknow':
            LED_Voltage(1)
        else:
            LED_Voltage(0)
            time.sleep(5)
            LED_Voltage(1)

In [ ]:
if __name__ == '__main__':
    EncodeingList = np.load('database/EncodeingList.npz')       # 导入数据库_______数据库所有图像编码集合
    EncodeingList = EncodeingList['arr_0']
    classNames = np.load('database/classNames.npz')             # 导入数据库_______数据库所有图像的名称（分类）
    classNames = classNames['arr_0']
    while 1:
        webcampicture(picture_path)                                 # 采集图片并保存
        faceRecognition(picture_path)                               # 人脸识别